## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [6]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [0]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
data = pd.read_csv('/gdrive/My Drive/AIML PG/R6_Neural_Networks_Internal_Lab/prices.csv')

### Check all columns in the dataset

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],inplace=True,axis=1)

In [10]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [17]:
data_new=data.iloc[0:1000]
data_new.shape

(1000, 5)

In [19]:
data_new['volume'] = data_new['volume']/1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Divide the data into train and test sets

In [48]:
from sklearn.model_selection import train_test_split
y_data = data_new['volume']
x_data = data_new.drop(columns='volume',inplace=False,axis=1)
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,train_size=0.7)
print('x_train {}'.format(x_train.shape))
print('y_train {}'.format(y_train.shape))
print('x_test {}'.format(x_test.shape))
print('y_test {}'.format(y_test.shape))

x_train (700, 4)
y_train (700,)
x_test (300, 4)
y_test (300,)


#### Convert Training and Test Data to numpy float32 arrays


In [49]:
x_train_array = x_train.values.astype('float32')
y_train_array = y_train.values.astype('float32')
x_test_array = x_test.values.astype('float32')
y_test_array = y_test.values.astype('float32')
print(type(x_train_array),x_train_array.dtype)
print(type(y_train_array),y_train_array.dtype)
print(type(x_test_array),x_test_array.dtype)
print(type(y_test_array),y_test_array.dtype)

<class 'numpy.ndarray'> float32
<class 'numpy.ndarray'> float32
<class 'numpy.ndarray'> float32
<class 'numpy.ndarray'> float32


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [50]:
from sklearn.preprocessing import StandardScaler
std_scl = StandardScaler()
x_train_array = std_scl.fit_transform(x_train_array)
x_test_array = std_scl.transform(x_test_array)
print(x_train_array.mean(),x_train_array.std())
print(x_test_array.mean(),x_test_array.std())

3.4059797e-10 1.0
0.094209634 1.0686355


## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape = (4,1))
b = tf.zeros(shape = (1))

2.Define a function to calculate prediction

In [0]:
def predict(x,w,b):
  xw = tf.matmul(x,w)
  y = tf.add(xw,b)
  return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual,y_predicted):
  diff = y_actual - y_predicted
  sq = tf.square(diff)
  avg = tf.reduce_mean(sq)
  return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def training(x,y_actual,w,b,learning_rate=0.01):
  with tf.GradientTape() as t:
    t.watch([w,b])
    current_prediction = predict(x, w, b)
    current_loss = loss(y_actual, current_prediction)
    
  #Calculate Gradients for Loss with respect to Weights and Bias
  dw, db = t.gradient(current_loss,[w, b])
    
  #Update Weights and Bias
  w = w - learning_rate*dw
  b = b - learning_rate*db
    
  return w, b


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [52]:
for epoch in range(1,101):
  w,b = training(x_train_array,y_train_array,w,b)
  print('Training loss at epoch {} is {}'.format(epoch,loss(y_test_array,predict(x_test_array,w,b))))
  if(epoch%5==0):
    print('Training loss at 5th epoch {} is {}'.format(epoch,loss(y_test_array,predict(x_test_array,w,b))))




Training loss at epoch 1 is Tensor("Mean_1:0", shape=(), dtype=float32)
Training loss at epoch 2 is Tensor("Mean_3:0", shape=(), dtype=float32)
Training loss at epoch 3 is Tensor("Mean_5:0", shape=(), dtype=float32)
Training loss at epoch 4 is Tensor("Mean_7:0", shape=(), dtype=float32)
Training loss at epoch 5 is Tensor("Mean_9:0", shape=(), dtype=float32)
Training loss at 5th epoch 5 is Tensor("Mean_10:0", shape=(), dtype=float32)
Training loss at epoch 6 is Tensor("Mean_12:0", shape=(), dtype=float32)
Training loss at epoch 7 is Tensor("Mean_14:0", shape=(), dtype=float32)
Training loss at epoch 8 is Tensor("Mean_16:0", shape=(), dtype=float32)
Training loss at epoch 9 is Tensor("Mean_18:0", shape=(), dtype=float32)
Training loss at epoch 10 is Tensor("Mean_20:0", shape=(), dtype=float32)
Training loss at 5th epoch 10 is Tensor("Mean_21:0", shape=(), dtype=float32)
Training loss at epoch 11 is Tensor("Mean_23:0", shape=(), dtype=float32)
Training loss at epoch 12 is Tensor("Mean_25:

### Get the shapes and values of W and b

In [84]:
print(w.shape)
print(w)
print(b.shape)
print(b)

(4, 1)
Tensor("sub_416:0", shape=(4, 1), dtype=float32)
(1,)
Tensor("sub_417:0", shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [82]:
with tf.Session() as sess:
  print(predict(x_test_array,w,b)[0].eval())

[4.425088]


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_data = pd.read_csv('/gdrive/My Drive/AIML PG/R6_Neural_Networks_Internal_Lab/Iris.csv')

In [90]:
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [91]:
iris_data = pd.get_dummies(data=iris_data,columns=['Species'])
iris_data.sample(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
46,47,5.1,3.8,1.6,0.2,1,0,0
65,66,6.7,3.1,4.4,1.4,0,1,0
95,96,5.7,3.0,4.2,1.2,0,1,0
50,51,7.0,3.2,4.7,1.4,0,1,0
83,84,6.0,2.7,5.1,1.6,0,1,0


### Splitting the data into feature set and target set

In [111]:
feature_set = iris_data[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']].values
target_set = iris_data[['Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica']].values
x_train,x_test,y_train,y_test = train_test_split(feature_set,target_set,train_size=0.8,shuffle=True)
print('x_train {}'.format(x_train.shape))
print('y_train {}'.format(y_train.shape))
print('x_test {}'.format(x_test.shape))
print('y_test {}'.format(y_test.shape))

x_train (120, 4)
y_train (120, 3)
x_test (30, 4)
y_test (30, 3)


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#model.add(tf.keras.layers.Dense(4,activation='sigmoid'))

model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [116]:
model.fit(x_train,y_train ,          
          validation_data=(x_test,y_test),
          epochs=100)

Train on 120 samples, validate on 30 samples
Epoch 1/100
120/120 [==============================] - 0s 2ms/sample - loss: 2.4748 - acc: 0.3167 - val_loss: 1.6155 - val_acc: 0.4000
Epoch 2/100
120/120 [==============================] - 0s 148us/sample - loss: 1.5081 - acc: 0.3167 - val_loss: 1.1633 - val_acc: 0.4000
Epoch 3/100
120/120 [==============================] - 0s 142us/sample - loss: 1.1470 - acc: 0.2833 - val_loss: 1.0905 - val_acc: 0.1000
Epoch 4/100
120/120 [==============================] - 0s 155us/sample - loss: 1.0650 - acc: 0.1750 - val_loss: 1.0678 - val_acc: 0.1333
Epoch 5/100
120/120 [==============================] - 0s 140us/sample - loss: 1.0342 - acc: 0.2000 - val_loss: 1.0396 - val_acc: 0.2000
Epoch 6/100
120/120 [==============================] - 0s 149us/sample - loss: 1.0055 - acc: 0.2750 - val_loss: 1.0146 - val_acc: 0.2333
Epoch 7/100
120/120 [==============================] - 0s 144us/sample - loss: 0.9926 - acc: 0.3583 - val_loss: 0.9805 - val_acc: 0.200

### Model Prediction

In [124]:
model.predict(x_test)

array([[0.8799512 , 0.09003569, 0.03001309],
       [0.02610794, 0.32576632, 0.6481257 ],
       [0.09128883, 0.51016474, 0.39854643],
       [0.02192776, 0.3861236 , 0.5919486 ],
       [0.8701685 , 0.09766419, 0.03216736],
       [0.8517844 , 0.11555807, 0.03265746],
       [0.09053541, 0.4424752 , 0.46698946],
       [0.01403194, 0.37113515, 0.6148329 ],
       [0.01757992, 0.500049  , 0.48237106],
       [0.01246521, 0.42619085, 0.5613439 ],
       [0.8953125 , 0.07931049, 0.02537695],
       [0.8238651 , 0.12438286, 0.05175207],
       [0.9581659 , 0.02763274, 0.01420142],
       [0.8814189 , 0.08901954, 0.02956152],
       [0.84491515, 0.11810874, 0.0369762 ],
       [0.11982769, 0.48141825, 0.39875412],
       [0.01660064, 0.3328023 , 0.6505971 ],
       [0.01080863, 0.40695226, 0.58223903],
       [0.85331815, 0.12065188, 0.02603004],
       [0.8123845 , 0.14202133, 0.04559426],
       [0.06567045, 0.5502237 , 0.38410592],
       [0.03422376, 0.26877013, 0.6970061 ],
       [0.

### Save the Model

In [0]:
model.save('iris_r6_nn_v1.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?